In [13]:
import numpy as np
import cv2
import sys
import os
import pickle
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS


# Directories
output_dir = os.path.join(RESULT_DIR, 'test_results')
tables_dir = os.path.join(output_dir, 'tables')

In [40]:
def get_table(dbase, level, ground_truth):
    results_csv = os.path.join(tables_dir, 
                               '{0}_full_results.csv'.format(dbase))

    # Columns
    suffix = '_{0}{1}'.format(level[0], ground_truth[0]) 
    columns = [k + suffix 
            for k in ['gt', 'tn', 'fp', 'fn', 'tp']]
    all_columns = ['video', 'frame', 'fold'] + columns
    cc = all_columns[4:]
    
    res_df = pd.read_csv(results_csv, index_col=0)
    res_df = res_df[all_columns]
    res_df = res_df.groupby([vid_fold]).sum().loc[:, cc]
    res_df['MCC'] = res_df.apply(lambda row: MCC(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)
    res_df['DIS'] = res_df.apply(lambda row: DIS(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)

    res_df['TPR'] =  res_df['tp'+suffix]/(res_df['tp'+suffix]+res_df['fn'+suffix])
    res_df['FPR'] =  res_df['fp'+suffix]/(res_df['fp'+suffix]+res_df['tn'+suffix])

    sum_df = res_df.sum()
    res_pure = res_df.copy()
    
    res_df.loc['Sum', :] = sum_df
    res_df.loc['Average', :] = res_pure.mean()
    res_df.loc['Std. deviation', :] = res_pure.std()
    res_df.loc['Median', :] = res_pure.median()
    res_df.loc['Overall', 'MCC'] = MCC(sum_df[cc[0]], sum_df[cc[1]],
                                       sum_df[cc[2]], sum_df[cc[3]])
    res_df.loc['Overall', 'DIS'] = DIS(sum_df[cc[0]], sum_df[cc[1]],
                                    sum_df[cc[2]], sum_df[cc[3]])
    res_df.loc['Overall', 'TPR'] = sum_df['tp'+suffix]/(sum_df['tp'+suffix]+sum_df['fn'+suffix])
    res_df.loc['Overall', 'FPR'] = sum_df['fp'+suffix]/(sum_df['fp'+suffix]+sum_df['tn'+suffix])
    # res_df = res_df[['tn'+suffix, 'fp'+suffix, 
    #                  'fn'+suffix, 'tp'+suffix, 'MCC', 'DIS',
    #                  'TPR', 'FPR']]
    return res_df


res_df = get_table('TCF_LMO', 'pixel', 'silhouette')
res_df

,tn_ps,fp_ps,fn_ps,tp_ps,MCC,DIS,TPR,FPR
video,,,,,,,,
1.0,7.940839e+06,8.376600e+04,5.956660e+05,1.838409e+06,0.812655,0.244942,0.755280,0.010439
2.0,1.054253e+07,2.676300e+05,1.297700e+05,7.821100e+04,0.274175,0.624442,0.376049,0.024757
3.0,1.003969e+07,0.000000e+00,2.191790e+05,3.393320e+05,0.771093,0.392435,0.607565,0.000000
4.0,7.694268e+06,8.711000e+03,6.261580e+05,1.959842e+06,0.834529,0.242136,0.757866,0.001131
5.0,8.177392e+06,1.061150e+05,1.949300e+04,2.660140e+06,0.969533,0.014732,0.992725,0.012810
...,...,...,...,...,...,...,...,...
Sum,6.366654e+08,1.313180e+07,2.858711e+06,1.210416e+07,33.648240,14.118216,45.556459,1.185261
Average,1.079094e+07,2.225728e+05,4.845273e+04,2.051553e+05,0.570309,0.239292,0.772143,0.020089
Std. deviation,8.507361e+05,3.914442e+05,1.157701e+05,4.734085e+05,0.284107,0.282928,0.289518,0.035373


In [68]:
dbase = 'LightGBM'
level = 'pixel'
ground_truth = 'silhouette'
vid_fold = 'video'

# dbases = ['DAOMC', 'ADMULT', 'MCBS_Both', 'mcDTSR', 'TCF_LMO',
#           'LightGBM_temporal','LightGBM_warp', 'mcDTSR_3', 'mcDTSR_4', 
#           'mcDTSR_5', 'mcDTSR_6']

dbases = ['DAOMC', 'ADMULT', 'MCBS_Both', 'mcDTSR', 'TCF_LMO','LightGBM_warp']

results = {d:get_table(d, level, ground_truth) for d in dbases}


# concatenate the DataFrames horizontally
df_concat = pd.concat(results.values(), axis=1)

# create a multi-level column index using the dictionary keys
columns = pd.MultiIndex.from_tuples([(key, col) for key in results.keys()
                                     for col in results[key].columns])

# assign the multi-level column index to the concatenated DataFrame
df_concat.columns = columns
df_concat = df_concat.loc[df_concat.index.isin(['Average', 'Std. deviation', 
                                                'Median', 'Overall']), :]

In [69]:
df_mcc = df_concat.loc[:, (slice(None), 'MCC')]
df_dis = df_concat.loc[:, (slice(None), 'DIS')]
df_tpr = df_concat.loc[:, (slice(None), 'TPR')]
df_fpr = df_concat.loc[:, (slice(None), 'FPR')]

df_mcc = df_mcc.droplevel(level=1, axis=1)
df_dis = df_dis.droplevel(level=1, axis=1)

In [70]:
df_mcc


,DAOMC,ADMULT,MCBS_Both,mcDTSR,TCF_LMO,LightGBM_warp
video,,,,,,
Average,0.423006,0.538049,0.002923,0.015659,0.570309,0.724486
Std. deviation,0.322120,0.255700,0.030121,0.094588,0.284107,0.314887
Median,0.454020,0.621913,-0.005587,-0.009858,0.606837,0.854752
Overall,0.698870,0.687827,0.010879,0.047076,0.612196,0.877175


In [71]:
print("\\begin{table}[ht!]\n\centering\n\\begin{adjustbox}{width=\columnwidth,center}\n")
print(df_mcc.to_latex())
print("\end{adjustbox}\n\end{table}")

\begin{table}[ht!]
\centering
\begin{adjustbox}{width=\columnwidth,center}

\begin{tabular}{lrrrrrr}
\toprule
{} &     DAOMC &    ADMULT &  MCBS\_Both &    mcDTSR &   TCF\_LMO &  LightGBM\_warp \\
video          &           &           &            &           &           &                \\
\midrule
Average        &  0.423006 &  0.538049 &   0.002923 &  0.015659 &  0.570309 &       0.724486 \\
Std. deviation &  0.322120 &  0.255700 &   0.030121 &  0.094588 &  0.284107 &       0.314887 \\
Median         &  0.454020 &  0.621913 &  -0.005587 & -0.009858 &  0.606837 &       0.854752 \\
Overall        &  0.698870 &  0.687827 &   0.010879 &  0.047076 &  0.612196 &       0.877175 \\
\bottomrule
\end{tabular}

\end{adjustbox}
\end{table}


/tmp/ipykernel_45737/3440309745.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_mcc.to_latex())


In [72]:
df_fpr

,DAOMC,ADMULT,MCBS_Both,mcDTSR,TCF_LMO,LightGBM_warp
,FPR,FPR,FPR,FPR,FPR,FPR
video,,,,,,
Average,0.005964,0.005662,0.015887,0.422749,0.020089,0.003934
Std. deviation,0.010496,0.010249,0.005755,0.071520,0.035373,0.006943
Median,0.000993,0.002035,0.015424,0.407143,0.007461,0.000771
Overall,0.005563,0.005277,0.015974,0.423404,0.020209,0.003748


In [37]:
print("\\begin{table}[ht!]\n\centering\n\\begin{adjustbox}{width=\columnwidth,center}\n")
print(df_mcc.to_latex())
print("\end{adjustbox}\n\end{table}")

\begin{table}[ht!]
\centering
\begin{adjustbox}{width=\columnwidth,center}

\begin{tabular}{lrrrrrrr}
\toprule
{} &     DAOMC &    ADMULT &  MCBS\_Both &    mcDTSR &   TCF\_LMO &  LightGBM\_temporal &  LightGBM\_warp \\
\midrule
Average        &  0.257386 &  0.347064 &   0.003831 &  0.017994 &  0.427878 &           0.351262 &       0.429087 \\
Std. deviation &  0.364513 &  0.375513 &   0.043044 &  0.098062 &  0.369830 &           0.320215 &       0.440135 \\
Median         &  0.000000 &  0.099813 &  -0.001342 &  0.000000 &  0.423431 &           0.324694 &       0.262905 \\
Overall        &  0.698870 &  0.687827 &   0.010879 &  0.047076 &  0.612196 &           0.534727 &       0.877175 \\
\bottomrule
\end{tabular}

\end{adjustbox}
\end{table}


/tmp/ipykernel_45737/3440309745.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_mcc.to_latex())


In [36]:
print("\\begin{table}[ht!]\n\centering\n\\begin{adjustbox}{width=\columnwidth,center}\n")
print(df_fpr.to_latex())
print("\end{adjustbox}\n\end{table}")

\begin{table}[ht!]
\centering
\begin{adjustbox}{width=\columnwidth,center}

\begin{tabular}{lrrrrrrr}
\toprule
{} &      DAOMC &     ADMULT &   MCBS\_Both &      mcDTSR &    TCF\_LMO & LightGBM\_temporal & LightGBM\_warp \\
{} &        FPR &        FPR &         FPR &         FPR &        FPR &               FPR &           FPR \\
\midrule
Average        &   0.411056 &   1.314350 &    1.994743 &    1.999831 &   1.812472 &          1.998819 &      0.681606 \\
Std. deviation &  11.987514 &  51.802410 &  108.296352 &  108.609395 &  85.403318 &        108.784228 &     24.936753 \\
Overall        &   0.205588 &   0.657281 &    0.997456 &    1.000000 &   0.906338 &          0.999494 &      0.340867 \\
Median         &   0.000000 &   1.000000 &    1.000000 &    1.000000 &   1.000000 &          1.000000 &      0.000000 \\
\bottomrule
\end{tabular}

\end{adjustbox}
\end{table}


/tmp/ipykernel_5825/1175515453.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_fpr.to_latex())


In [72]:
# Create the style object
df_style = df_mcc.style.apply(
    lambda x: ['font-weight: bold' if v == x[x.index.max()] else '' for v in x],
    axis=1
).apply(
    lambda x: ['font-weight: bold' if v == x[x.index.min()] else '' for v in x],
    axis=1, subset=pd.IndexSlice['Std. deviation', :]
)

# Convert to LaTeX
latex_table = df_style.to_latex()

print(latex_table)


\begin{tabular}{lrrrrrrr}
 & DAOMC & ADMULT & MCBS_Both & mcDTSR & TCF_LMO & LightGBM_temporal & LightGBM_warp \\
video &  &  &  &  &  &  &  \\
Average & 0.380501 & 0.457119 & 0.000558 & \font-weightbold -0.014802 & 0.030048 & 0.010230 & 0.623659 \\
Std. deviation & 0.283957 & \font-weightbold 0.201435 & 0.027383 & \font-weightbold 0.043969 & 0.085761 & 0.045464 & 0.270707 \\
Overall & 0.604219 & 0.584970 & 0.006895 & \font-weightbold -0.014536 & 0.064795 & 0.052447 & 0.772486 \\
Median & 0.460846 & 0.477773 & -0.007479 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & 0.721401 \\
\end{tabular}

